# 16. Bursty gene expression

<hr>

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade biocircuits line_profiler multiprocess watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
# ------------------------------

try:
    import multiprocess
except:
    import multiprocessing as multiprocess

import tqdm

import numpy as np
import scipy.stats as st
import numba

import biocircuits

# Plotting modules
import iqplot
import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

# Line profiler (can install with conda install line_profiler)
%load_ext line_profiler

Loading BokehJS ...

<hr>

**Design principle**

- Bursty gene expression enables cells to regulate the mean and cell-cell variability of protein levels by controlling burst frequency and burst size.

**Concepts**

- Master equations as gain-loss equations for probability.
- Master equations can equivalently be defined in terms of a set of moves and associated propensities and updates.
- Sampling out of probability distributions is a useful way to computationally determine their properties.
- The Gillespie algorithm enables sampling out of probability distributions described by master equations.
- Gene expression occurs in bursts.

**Techniques**

- Manipulating master equations
- Probability stories (Poisson, Geometric, and Negative Binomial)
- Gillespie algorithm, a.k.a. stochastic simulation algorithm)

<hr>

In the last lesson, we laid out two objectives. First, we aimed to identify the sources of noise and how to characterize them. We summarized statistics about the probability distribution of copy numbers by considering the mean and coefficient of variation. The mean ends up following the same deterministic dynamics we have been considering thus far in the course. The coefficient of variation is a measure of noise, or relative departure from the mean behavior. Now, we will look at the dynamics of the entire probability distribution of copy number, $P(n,t)$.

## Master equations

We will use **master equations** to describe the dynamics of $P(n,t)$.  Generally, a master equation is a loss-gain equation for probabilities of states governed by a Markov process. (To see where it gets its name, you can read [this amusing blog post by Nicole Yunger Halpern](https://quantumfrontiers.com/2015/06/21/hello-my-name-is-quantum-master-equation/).) In our case, the "state" is the set of copy numbers of molecular species of interest. The values of $n$ are discrete, so we have a separate differential equation for each $P(n,t)$.  Specifically,

\begin{align}
\frac{\mathrm{d}P(n,t)}{\mathrm{d}t} = \sum_{n'}\left[W(n\mid n') P(n',t) - W(n'\mid n)P(n,t)\right].
\end{align}

Here, $W(n\mid n')$ is the transition probability per unit time of going from $n'$ to $n$.

The master equation makes sense by inspection and appears simple. The nuance lies in the definition of the transition rates, $W(n\mid n')$. There is also the computational difficulty that $n$ can be very large. In general, solving the master equation is difficult and is usually intractable analytically.  Therefore, we *sample* out of the distribution. That is, we can draw many samples of values of $n$ at time points $t$ that are distributed according to the probability distribution that solves the master equation. We can then plot histograms to get an approximate plot of the probability distribution.  We can also use the samples to compute moments, giving us estimates of the mean and variance. Generating the samples from the *differential* master equation is done using a *Gillespie algorithm*, also known as a *stochastic simulation algorithm*, or SSA, which we will discuss later in this chapter.

Note in using a master equation we are assuming that events that change state (that is, that change the copy number of one or more species) are **Poisson processes**. In this context, a [Poisson process](https://en.wikipedia.org/wiki/Poisson_point_process) may be thought of as a series of well-defined, separate events that occur randomly, without memory of what has occurred before. This is often the case for things like molecular collisions. Implicit in the assumption that state changes are modeled as Poisson processes is that the binding or dissociation (or any other event) happens essentially instantaneously with well-defined pauses between them. This is yet another instance of where a separation of time scales is important.

## The Master equation for unregulated gene expression

As we have seen, unregulated gene expression is described by the macroscale equation

\begin{align}
\frac{\mathrm{d}n}{\mathrm{d}t} = \beta - \gamma n.
\end{align}

To translate this into stochastic dynamics, we write the production rate as

\begin{align}
\text{production rate} = \beta \delta_{n',n-1},
\end{align}

where we have used the Kronecker delta,

\begin{align}
\delta_{ij} = \left\{\begin{array}{lll}
1 && i = j \\
0 && i\ne j.
\end{array} \right.
\end{align}

This says that if our current copy number is $n-1$, the probability that it moves to $n$ in unit time is $\beta$.  Similarly, the decay rate is

\begin{align}
\text{decay rate} = \gamma (n+1) \delta_{n',n+1}.
\end{align}

This says that if we have $n+1$ molecules, the probability that the copy number moves to $n$ in unit time is $\gamma(n+1)$.  Thus, we have

\begin{align}
W(n\mid n') = \beta \delta_{n',n-1} + \gamma (n+1)\delta_{n',n+1}.
\end{align}

We can then write our master equation as

\begin{align}
\frac{\mathrm{d}P(n,t)}{\mathrm{d}t} = \beta P(n-1,t) + \gamma(n+1)P(n+1,t)
- \beta P(n,t) - \gamma n P(n,t),
\end{align}

where we define $P(n<0,t) = 0$.

This is a large set of ODEs, and as mentioned in the previous section, solving for $P(n,t)$ is nontrivial and is usually done by Gillespie algorithm. Instead, let's look for a steady state solution of this system of ODEs; i.e., let's find $P(n)$ that satisfies

\begin{align}
\frac{\mathrm{d}P}{\mathrm{d}t} = 0.
\end{align}

To solve this, we will start with the case where $n=0$, noting that $P(n<0) = 0$.

\begin{align}
\gamma P(1) - \beta P(0) = 0 \;\;\Rightarrow P(1)\;\; = \frac{\beta}{\gamma}P(0).
\end{align}

We can write the expression for the case where $n=1$.

\begin{align}
\beta P(0) + 2 \gamma P(2) - \beta P(1) - \gamma P(1) = 0 \;\;\Rightarrow\;\; P(2) = \frac{1}{2\gamma}\left(\beta P(1) + \gamma P(1) + \beta P(0)\right).
\end{align}

We can use the fact that $P(1) = \beta P(0) / \gamma$ to simplify this expression to

\begin{align}
P(2) = \frac{1}{2}\left(\frac{\beta}{\gamma}\right)^2\,P(0).
\end{align}

We do it again for $n = 2$.

\begin{align}
\beta P(1) + 3 \gamma P(3) - \beta P(2) - 2\gamma P(2) = 0 \;\;\Rightarrow\;\; P(3) = \frac{1}{3\gamma}\left(\beta P(2) - 2\gamma P(2) + \beta P(1)\right).
\end{align}

Using our expressions for $P(1)$ and $P(2)$, we can rearrange this expression to write $P(3)$ in terms of $P(0)$, giving

\begin{align}
P(3) = \frac{1}{6}\left(\frac{\beta}{\gamma}\right)^3\,P(0).
\end{align}

We can continue this process over and over again such that for any $n$, we have

\begin{align}
P(n) = \frac{1}{n!}\left(\frac{\beta}{\gamma}\right)^n P(0).
\end{align}

To solve for $P(n)$, then, we need to find $P(0)$. We can use the fact that the probability distribution must be normalized;

\begin{align}
\sum_{n=0}^\infty P(n) = P(0) \sum_{n=0}^\infty \frac{1}{n!}\left(\frac{\beta}{\gamma}\right)^n = P(0)\mathrm{e}^{\beta/\gamma} = 1,
\end{align}

where we have noted that the sum is the [Taylor series](https://en.wikipedia.org/wiki/Taylor_series) expansion of the exponential function. Thus, we have $P(0) = \mathrm{e}^{-\beta/\gamma}$, and we have our steady state probability mass function,

\begin{align}
P(n) = \frac{1}{n!}\left(\frac{\beta}{\gamma}\right)^n \mathrm{e}^{-\beta/\gamma}.
\end{align}

We recognize this as the PMF of the [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution). We could perhaps have guessed this from the "story" of the Poisson distribution.

>Rare events occur with a rate $\lambda$ per unit time. There is no "memory" of previous events; i.e., that rate is independent of time.  The probability that $k$ events occur in unit time is Poisson distributed.

The story of mRNA production matches this story. The event here is the production of a gene product.  We need these events to happen before decay. Apparently, the rate $\lambda$ is in this case $\beta/\gamma$.

It is useful also to consider the moments of the Poisson distribution and compute useful summary statistics of the distribution.

\begin{align}
&\text{mean} = \langle n \rangle = \frac{\beta}{\gamma}, \\[1em]
&\text{variance} = \sigma^2 = \langle n^2 \rangle - \langle n \rangle^2 = \frac{\beta}{\gamma}, \\[1em]
&\text{noise} = \text{coefficient of variation} = \eta = \frac{\sigma}{\langle n \rangle} = \sqrt{\frac{\gamma}{\beta}}, \\[1em]
&\text{Fano factor} = F = \frac{\sigma^2}{\langle n \rangle} = 1.
\end{align}


## Dynamics of the moments

From the master equation, we can derive an ODE describing the dynamics of the mean copy number of time, $\langle n(t) \rangle$.  To do this, we multiply both sides of the master equations by $n$ and then sum over $n$.

\begin{align}
&\sum_{n=0}^\infty n \left[
\frac{\mathrm{d}P(n,t)}{\mathrm{d}t} = \beta P(n-1,t) + \gamma(n+1)P(n+1,t)
- \beta P(n,t) - \gamma n P(n,t)\right] \nonumber \\[1em]
&= \frac{d\langle n \rangle}{\mathrm{d}t} = \beta \sum_{n=0}^\infty n P(n-1,t) + \gamma \sum_{n=0}^\infty n(n+1)P(n+1,t) - \beta\langle n \rangle - \gamma \langle n^2 \rangle,
\end{align}

where we have used the facts that

\begin{align}
\langle n \rangle &= \sum_{n=0}^\infty n P(n,t), \\[1em]
\langle n^2 \rangle &= \sum_{n=0}^\infty n^2 P(n,t).
\end{align}

We have two sums left to evaluate.

\begin{align}
\sum_{n=0}^\infty n P(n-1,t) = \sum_{n=0}^\infty (n+1)P(n,t) = \langle n \rangle + 1,
\end{align}

and

\begin{align}
\sum_{n=0}^\infty n(n+1)P(n+1,t) = \sum_{n=0}^\infty n(n-1)P(n,t)
= \langle n^2 \rangle - \langle n\rangle.
\end{align}

Thus, we have

\begin{align}
\frac{d\langle n \rangle}{\mathrm{d}t} &= \beta(\langle n \rangle + 1) + \gamma (\langle n^2 \rangle - \langle n\rangle) - \beta\langle n \rangle - \gamma \langle n^2 \rangle \nonumber \\[1em]
&= \beta - \gamma \langle n \rangle,
\end{align}

which is precisely the macroscale ODE we are used to. It is now clear that it describes the mean of the full probability distribution. The assumptions behind our ODE models are now also clear. They describe the dynamics of the expectation value of copy number (or concentration when divided by the total volume) whose probability distribution is described by a master equation.

## Sampling out of master equations

We have managed to derive the steady state solution to a master equation, but we would like to know the full dynamics. In nearly all cases we may come across in modeling biological circuits, an exact analytical expression for $P(n;t)$ is not accessible. We therefore turn to **sampling** out of $P(n;t)$, a concept we will first consider with simpler distributions.

### Sampling out of probability distributions

Sampling out of a distribution involves using a random number generator to *simulate* the **story** that generates the distribution. So, if you know the story and you have a computer handy, you can sample to be able to get a plot of your distribution, though you will not get the analytical form.

Let's demonstrate this with the Binomial distribution. We know that if we flip a coin that has probability $p$ of landing heads, the number of heads in $n$ flips is Bionomially distributed. Imagine for a moment that we did not know that, but will instead take to sampling using the story. We will take $n = 25$ and $p = 0.25$ and compute $P(h \mid n, p)$, the probability of getting $h$ heads in $n$ flips, each with probability $p$ of landing heads. We will draw 10, 30, 100, and 300 samples by simulating the story of the Binomial distribution and plot the result obtained by sampling along with the expected Binomial distribution.

In [2]:
def simulate_coinflips(n, p, size=1):
    """
    Simulate n_samples sets of n coin flips with prob. p of heads.
    """
    n_heads = np.empty(size, dtype=np.int64)
    for i in range(size):
        n_heads[i] = np.sum(np.random.random(size=n) < p)
    return n_heads


size = (30, 100, 1000, 10000)
n = 25
p = 0.25
h_plot = np.arange(26)
theor_dist = st.binom.pmf(h_plot, n, p)

plots = []
for n_samp in size:
    plot = bokeh.plotting.figure(
        plot_height=200,
        plot_width=300,
        x_axis_label="h",
        y_axis_label="P(h)",
        title=f"{n_samp} samples",
    )
    h = simulate_coinflips(n, p, size=n_samp)
    plot.circle(h_plot, theor_dist)
    plot.circle(
        np.arange(h.max() + 1), np.bincount(h) / n_samp, color="orange"
    )
    plots.append(plot)

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=2))

As we can see, if we *sample* out of the probability distribution, we can approximately calculate the actual distribution. If we sample enough, the approximation is very good.

Sampling is such a powerful strategy that highly efficient algorithms with convenient APIs have been developed to sample out of named probability distributions. For example, we could have used `np.random.binom()` as a drop-in (and _much_ more efficient) replacement for the `simulate_coinflips()` function above.

We will use the same strategy for solving master equations. We will find a way to *sample* out of the distribution that is governed by the master equation. This technique was pioneered by Dan Gillespie in the last 70s. For that reason, these sampling techniques are often called **Gillespie simulations**. The algorithm is sometimes referred to as a **stochastic simulation algorithm**, or SSA.

### Example: Unregulated gene expression

Here will explore how this algorithm works by looking at simple production of a protein. We will include the production of RNA to demonstrate how this is done with a master equation that has more than one species.

For simple protein production, we have the following reactions.

\begin{align}
\text{DNA} \rightarrow \text{mRNA} \rightarrow \text{protein}
\end{align}

#### Macroscale equations

As we've seen before, the deterministic dynamics, which describe mean concentrations over a large population of cells, are described by the ODEs

\begin{align}
\frac{\mathrm{d}m}{\mathrm{d}t} &= \beta_m - \gamma_m m, \\[1em]
\frac{\mathrm{d}p}{\mathrm{d}t} &= \beta_p m - \gamma_p p.
\end{align}

The same equations should hold if $m$ and $p$ represent the mean *numbers* of cells; we would just have to appropriately rescale the constants. Assuming the $m$ and $p$ are now numbers (so we are not free to pick their units), we can nondimensionalize using $\gamma_m$ to nondimensionalize time. This leads to redefinition of parameters and variables

\begin{align}
&\beta_m/\gamma_m \to \beta_m, \\[1em]
&\beta_p/\gamma_m \to \beta_p, \\[1em]
&\gamma_m t \to t.
\end{align}

The dimensionless equations are

\begin{align}
\frac{\mathrm{d}m}{\mathrm{d}t} &= \beta_m - m, \\[1em]
\frac{\mathrm{d}p}{\mathrm{d}t} &= \beta_p m - \gamma p,
\end{align}

with $\gamma = \gamma_p/\gamma_m$.

#### The Master equation

We can write a master equation for these dynamics.  In this case, each state is defined by an mRNA copy number $m$ and a protein copy number $p$.  So, we will write a master equation for $P(m, p, t)$.

\begin{align}
\frac{\mathrm{d}P(m,p,t)}{\mathrm{d}t} &= \beta_m P(m-1,p,t) + (m+1)P(m+1,p,t) - \beta_m P(m,p,t) - mP(m,p,t) \nonumber \\[1em]
&+\beta_p mP(m,p-1,t) + \gamma (p+1)P(m,p+1,t) - \beta_p mP(m,p,t) - \gamma p P(m,p,t).
\end{align}

We implicitly define $P(m, p, t) = 0$ if $m < 0$ or $p < 0$. This is the master equation we will sample from using the stochastic simulation algorithm (SSA), also known as the Gillespie algorithm.

### The Gillespie algorithm

The Gillespie algorithm is a way to sample the story behind a master equation, as will become clear momentarily as we work through the algorithm and introduce some terminology.

The transition probabilities are also called **propensities** in the context of stochastic simulation. The propensity for a given transition, say indexed $i$, is denoted as $a_i$. The equivalence to notation we introduced for master equations is that if transition $i$ results in the change of state from $n'$ to $n$, then $a_i = W(n\mid n')$.

To cast this problem for a Gillespie simulation, we can write each change of state (moving either the copy number of mRNA or protein up or down by 1 in this case) and their respective propensities.

\begin{align}
\begin{array}{ll}
\text{reaction, }r_i & \text{propensity, } a_i \\
m \rightarrow m+1,\;\;\;\; & \beta_m \\[0.3em]
m \rightarrow m-1, \;\;\;\; & m\\[0.3em]
p \rightarrow p+1, \;\;\;\; & \beta_p m \\[0.3em]
p \rightarrow p-1, \;\;\;\; & \gamma p.
\end{array}
\end{align}

We will not carefully prove that the Gillespie algorithm samples from the probability distribution governed by the master equation, but will state the principles behind it. The basic idea is that events (such as those outlined above) are rare, discrete, separate events. I.e., each event is an arrival of a Poisson process. The Gillespie algorithm starts with some state, $(m_0,p_0)$. Then a state change, *any* state change, will happen in some time $\Delta t$ that has a certain probability distribution (which we will show is Exponential momentarily). The probability that the state change that happens is reaction $j$ is proportional to $a_j$. That is to say, state changes with high propensities are more likely to occur. Thus, choosing which of the $n$ state changes happens in $\Delta t$ is a matter of drawing an integer $j$ in $[1,n]$ where the probability of drawing $j$ is

\begin{align}
\frac{a_j}{\sum_i a_i}.
\end{align}

Now, how do we determine how long the state change took?  The probability density function describing that a *given* state change $i$ takes place in time $t$ is

\begin{align}
P(t\mid a_i) = a_i\, \mathrm{e}^{-a_i t},
\end{align}

since the time it takes for arrival of a [Poisson process](https://en.wikipedia.org/wiki/Poisson_point_process) is [Exponentially distributed](https://en.wikipedia.org/wiki/Exponential_distribution). The probability that it has *not* arrived in time $\Delta t$ is the probability that the arrival time is greater than $\Delta t$, given by the complementary cumulative distribution function for the Exponential distribution.

\begin{align}
P(t > \Delta t\mid a_i) = \int_{\Delta t}^\infty \mathrm{d}t\,P(t\mid a_i) = \mathrm{e}^{-a_i \Delta t}.
\end{align}

Now, say we have $n$ processes that arrive in time $t_1, t_2, \ldots$.  The probability that *none* of them arrive before $\Delta t$ is

\begin{align}
P(t_1 > \Delta t, t_2 > \Delta t, \ldots) &=
P(t_1 > \Delta t) P(t_2 > \Delta t) \cdots =
\prod_i \mathrm{e}^{-a_i \Delta t} 
= \mathrm{exp}\left[-\Delta t \sum_i a_i\right].
\end{align}

This is the same as the probability of a single Poisson process with $a = \sum_i a_i$ not arriving before $\Delta t$. So, the probability that it *does* arrive in $\Delta t$ is Exponentially distributed with mean $\left(\sum_i a_i\right)^{-1}$.

So, we know how to choose a state change and we also know how long it takes.
The Gillespie algorithm then proceeds as follows.

1. Choose an initial condition, e.g., $m = p = 0$.
2. Calculate the propensity for each of the enumerated state changes.  The propensities may be functions of $m$ and $p$, so they need to be recalculated for every $m$ and $p$ we encounter.
3. Choose how much time the reaction will take by drawing out of an exponential distribution with a mean equal to $\left(\sum_i a_i\right.)^{-1}$.  This means that a change arises from a Poisson process.
4. Choose what state change will happen by drawing a sample out of the discrete distribution where $P_i = \left.a_i\middle/\left(\sum_i a_i\right)\right.$.  In other words, the probability that a state change will be chosen is proportional to its propensity.
5. Increment time by the time step you chose in step 3.
6. Update the states according to the state change you choose in step 4.
7. If $t$ is less than your pre-determined stopping time, go to step 2. Else stop.

Gillespie proved that this algorithm samples the probability distribution described by the master equation in his seminal papers in [1976](https://doi.org/10.1016/0021-9991(76)90041-3) and [1977](http://doi.org/10.1021/j100540a008). (We recommend reading the latter.) You can also read a concise discussion of how the algorithm samples the master equation in [section 4.2 of Del Vecchio and Murray](http://www.cds.caltech.edu/~murray/books/AM08/pdf/bfs-stochastic_14Sep14.pdf).

### Coding up a Gillespie simulation

To code up the Gillespie simulation of the simple gene expression example, we first make an an array that gives the changes in the counts of $m$ and $p$ for each of the four reactions. This is a way of encoding the updates in the particle counts that we get from choosing the respective state changes.

In [3]:
# Column 0 is change in m, column 1 is change in p
simple_update = np.array(
    [
        [1, 0],   # Make mRNA transcript
        [-1, 0],  # Degrade mRNA
        [0, 1],   # Make protein
        [0, -1],  # Degrade protein
    ],
    dtype=np.int64,
)

Next, we make a function that updates the array of propensities for each of the four reactions. We update the propensities (which are passed into the function as an argument) instead of instantiating them and returning them to save on memory allocation while running the code. It has the added benefit that it forces you to keep track of the indices corresponding to the update matrix. This helps prevent bugs. It will naturally be a function of the current population of molecules. It may in general also be a function of time, so we explicitly allow for time dependence (even though we will not use it in this simple example) as well.

In [4]:
def simple_propensity(propensities, population, t, beta_m, beta_p, gamma):
    """Updates an array of propensities given a set of parameters
    and an array of populations.
    """
    # Unpack population
    m, p = population

    # Update propensities
    propensities[0] = beta_m  # Make mRNA transcript
    propensities[1] = m  # Degrade mRNA
    propensities[2] = beta_p * m  # Make protein
    propensities[3] = gamma * p  # Degrade protein

### Making a draw

Finally, we write a general function that draws a choice of reaction and the time interval for that reaction.  This is the heart of the Gillespie algorithm, so we will take some time to discuss speed.  First, to get the time interval, we sample a random number from an exponential distribution with mean $\left(\sum_i a_i\right)^{-1}$.  This is easily done using the `np.random.exponential()` function.  

Next, we have to select which reaction will take place.  This amounts to drawing a sample over the discrete distribution where $P_i = a_i\left(\sum_i a_i\right)^{-1}$, or the probability of each reaction is proportional to its propensity.  This can be done using `scipy.stats.rv_discrete`, which allows specification of an arbitrary discrete distribution.  We will write a function to do this.

In [5]:
def sample_discrete_scipy(probs):
    """Randomly sample an index with probability given by probs."""
    return st.rv_discrete(values=(range(len(probs)), probs)).rvs()

This is a nice one-liner, but is it fast? There may be significant overhead in setting up the `scipy.stats` discrete random variable object to sample from each time. Remember, we can't just do this once because the array `probs` changes with each step in the SSA because the propensities change. We will therefore write a less elegant, but maybe faster way of doing it.  

Another way to sample the distribution is to generate a uniformly distributed random number $q$, with $0 < q < 1$ and return the value $j$ such that

\begin{align}
\sum_{i=0}^{j-1} p_i < q < \sum_{i=0}^{j}p_i.
\end{align}

We'll code this up.

In [6]:
def sample_discrete(probs):
    """Randomly sample an index with probability given by probs."""
    # Generate random number
    q = np.random.rand()
    
    # Find index
    i = 0
    p_sum = 0.0
    while p_sum < q:
        p_sum += probs[i]
        i += 1
    return i - 1

Now let's compare the speeds using the `%timeit` magic function. This is a useful tool to help diagnose slow spots in your code.

In [7]:
# Make dummy probs
probs = np.array([0.1, 0.3, 0.4, 0.05, 0.15])

print('Result from scipy.stats:')
%timeit sample_discrete_scipy(probs)

print('\nResult from hand-coded method:')
%timeit sample_discrete(probs)

Result from scipy.stats:
668 µs ± 31.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Result from hand-coded method:
1.22 µs ± 27.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Wow!  The less concise method is a couple of orders of magnitude faster! So, we will ditch using `scipy.stats`, and use our hand-built sampler instead. (You can find a much more thorough discussion on speed considerations in Gillespie simulations in Appendix B.)

Now we can write a function to do our draws.

In [8]:
def gillespie_draw(propensity_func, propensities, population, t, args=()):
    """
    Draws a reaction and the time it took to do that reaction.
    
    Parameters
    ----------
    propensity_func : function
        Function with call signature propensity_func(population, t, *args)
        used for computing propensities. This function must return
        an array of propensities.
    propensities : ndarray
        Propensities for each reaction as a 1D Numpy array.
    population : ndarray
        Current population of particles
    t : float
        Value of the current time.
    args : tuple, default ()
        Arguments to be passed to `propensity_func`.
        
    Returns
    -------
    rxn : int
        Index of reaction that occured.
    time : float
        Time it took for the reaction to occur.
    """
    # Compute propensities
    propensity_func(propensities, population, t, *args)
    
    # Sum of propensities
    props_sum = propensities.sum()
    
    # Compute next time
    time = np.random.exponential(1.0 / props_sum)
    
    # Compute discrete probabilities of each reaction
    rxn_probs = propensities / props_sum
    
    # Draw reaction from this distribution
    rxn = sample_discrete(rxn_probs)
    
    return rxn, time

### Gillespie time stepping

Now we are ready to write our mainA loop. We will only keep the counts at pre-specified time points. This saves on RAM, and we really only care about the values at given time points anyhow.

Note that this function is generic. All we need to specify our system is the following.

* A function to compute the propensities
* How the updates for a given reaction are made
* Initial population

Additionally, we specify necessary parameters, an initial condition, and the time points at which we want to store our samples. So, providing the propensity function and update are analogous to providing the time derivatives when using `scipy.integrate.odeint()`.

In [9]:
def gillespie_ssa(propensity_func, update, population_0, time_points, args=()):
    """
    Uses the Gillespie stochastic simulation algorithm to sample
    from probability distribution of particle counts over time.
    
    Parameters
    ----------
    propensity_func : function
        Function of the form f(params, t, population) that takes the current
        population of particle counts and return an array of propensities
        for each reaction.
    update : ndarray, shape (num_reactions, num_chemical_species)
        Entry i, j gives the change in particle counts of species j
        for chemical reaction i.
    population_0 : array_like, shape (num_chemical_species)
        Array of initial populations of all chemical species.
    time_points : array_like, shape (num_time_points,)
        Array of points in time for which to sample the probability
        distribution.
    args : tuple, default ()
        The set of parameters to be passed to propensity_func.        

    Returns
    -------
    sample : ndarray, shape (num_time_points, num_chemical_species)
        Entry i, j is the count of chemical species j at time
        time_points[i].
    """

    # Initialize output
    pop_out = np.empty((len(time_points), update.shape[1]), dtype=np.int64)

    # Initialize and perform simulation
    i_time = 1
    i = 0
    t = time_points[0]
    population = population_0.copy()
    pop_out[0, :] = population
    propensities = np.zeros(update.shape[0])
    while i < len(time_points):
        while t < time_points[i_time]:
            # draw the event and time step
            event, dt = gillespie_draw(
                propensity_func, propensities, population, t, args
            )

            # Update the population
            population_previous = population.copy()
            population += update[event, :]

            # Increment time
            t += dt

        # Update the index
        i = np.searchsorted(time_points > t, True)

        # Update the population
        pop_out[i_time : min(i, len(time_points))] = population_previous

        # Increment index
        i_time = i

    return pop_out

### Running and parsing results

We can now run a set of SSA simulations and plot the results.  We will run 100 trajectories and store them, using $\beta_p = \beta_m = 10$ and $\gamma = 0.4$. We will also use the nifty package `tqdm` to give a progress bar so we know how long it is taking.

In [10]:
# Specify parameters for calculation
args = (10.0, 10.0, 0.4)
time_points = np.linspace(0, 50, 101)
population_0 = np.array([0, 0], dtype=int)
size = 100

# Seed random number generator for reproducibility
np.random.seed(42)

# Initialize output array
samples = np.empty((size, len(time_points), 2), dtype=np.int64)

# Run the calculations
for i in tqdm.tqdm(range(size)):
    samples[i, :, :] = gillespie_ssa(
        simple_propensity, simple_update, population_0, time_points, args=args
    )

100%|██████████████████████████████████████████████████████████████████████| 100/100 [00:25<00:00,  3.97it/s]


We now have our samples, so we can plot the trajectories. For visualization, we will plot every trajectory as a thin blue line, and then the average of the trajectories as a thick orange line.

In [11]:
# Set up plots
plots = [
    bokeh.plotting.figure(
        plot_width=300,
        plot_height=200,
        x_axis_label="dimensionless time",
        y_axis_label="number of mRNAs",
    ),
    bokeh.plotting.figure(
        plot_width=300,
        plot_height=200,
        x_axis_label="dimensionless time",
        y_axis_label="number of proteins",
    ),
]

# Plot trajectories and mean
for i in [0, 1]:
    for x in samples[:, :, i]:
        plots[i].line(
            time_points, x, line_width=0.3, alpha=0.2, line_join="bevel"
        )
    plots[i].line(
        time_points,
        samples[:, :, i].mean(axis=0),
        line_width=6,
        color="orange",
        line_join="bevel",
    )

# Link axes
plots[0].x_range = plots[1].x_range

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=2))

We can also compute the steady state properties by considering the end of the simulation.  The last 50 time points are at steady state, so we will average over them.

In [12]:
print("mRNA mean copy number =", samples[:, -50:, 0].mean())
print("protein mean copy number =", samples[:, -50:, 1].mean())
print("\nmRNA variance =", samples[:, -50:, 0].std() ** 2)
print("protein variance =", samples[:, -50:, 1].std() ** 2)
print("\nmRNA noise =", samples[:, -50:, 0].std() / samples[:, -50:, 0].mean())
print(
    "protein noise =", samples[:, -50:, 1].std() / samples[:, -50:, 1].mean()
)

mRNA mean copy number = 10.0748
protein mean copy number = 251.6994

mRNA variance = 10.26200496
protein variance = 2145.33703964

mRNA noise = 0.317965262817717
protein noise = 0.18402023679851773


Finally, we can compute the steady state probability distributions. To plot them, we plot the empirical cumulative distribution function (ECDF) from the sampling.  The theoretical distribution for mRNA is Poisson, which is overlayed in orange.

In [13]:
# mRNA ECDF
p_m = iqplot.ecdf(
    samples[:, -50:, 0].flatten(),
    plot_width=300,
    plot_height=200,
    x_axis_label="mRNA copy number",
    style="staircase"
)

# Theoretial mRNA CDF (Poisson)
p_m.circle(
    np.arange(25), st.poisson.cdf(np.arange(25), args[0]), color="orange"
)

# protein ECDF
p_p = iqplot.ecdf(
    samples[:, -50:, 1].flatten(),
    plot_width=300,
    plot_height=200,
    x_axis_label="protein copy number",
    style="staircase",
)

bokeh.io.show(bokeh.layouts.gridplot([p_m, p_p], ncols=2))

As we expect, the mRNA copy number matches the Poisson distribution. We also managed to get a distribution for the protein copy number that we could plot. 

You now have the basic tools for doing Gillespie simulations. You just need to code the propensity function and the update, and you're on your way! (Note, though, that this only works for Gillespie simulations where the states are defined by particle counts, which should suffice for this course.)

## Implementation in the biocircuits package

This algorithm is implemented in the biocircuits package. A significant speed boost is achieved by [just-in-time compliation](https://en.wikipedia.org/wiki/Just-in-time_compilation) using [Numba](http://numba.pydata.org). To utilize this feature, you need to just-in-time compile (JIT) your propensity function. You can insist that everything is compiled (and therefore skips the comparably slow Python interpreter) by using the <font style="font-family: monospace;">@numba.njit</font> decorator. In many cases, like in this one for simple gene expression, that is all you have to do.

In [14]:
@numba.njit
def simple_propensity(propensities, population, t, beta_m, beta_p, gamma):
    """Updates an array of propensities given a set of parameters
    and an array of populations.
    """
    # Unpack population
    m, p = population

    # Update propensities
    propensities[0] = beta_m      # Make mRNA transcript
    propensities[1] = m           # Degrade mRNA
    propensities[2] = beta_p * m  # Make protein
    propensities[3] = gamma * p   # Degrade protein

You can also utilize more threads and do the Gillespie simulations in parallel if you like using the `n_threads` keyword argument (making sure that `n_threads` does not exceed the number of cores on your machine). Note that the total number of samples is `n_threads * size`. Because of the speed boosts, we will take more samples using the biocircuits implementation.

In [15]:
samples = biocircuits.gillespie_ssa(
    simple_propensity,
    simple_update,
    population_0,
    time_points,
    size=2000,
    args=args,
    n_threads=2,
    progress_bar=False,
)

We now have 4000 trajectories, and we can again make the plots as we did before. With so many trajectories, though, we should not show them all, since there would be too many glyphs on the plot. We therefore thin out the samples for plotting, taking only every 100th trajectory.

In [16]:
# Set up plots
plots = [
    bokeh.plotting.figure(
        plot_width=300,
        plot_height=200,
        x_axis_label="dimensionless time",
        y_axis_label="number of mRNAs",
    ),
    bokeh.plotting.figure(
        plot_width=300,
        plot_height=200,
        x_axis_label="dimensionless time",
        y_axis_label="number of proteins",
    ),
]

# Plot trajectories and mean
for i in [0, 1]:
    for x in samples[::100, :, i]:
        plots[i].line(
            time_points, x, line_width=0.3, alpha=0.2, line_join="bevel"
        )
    plots[i].line(
        time_points,
        samples[:, :, i].mean(axis=0),
        line_width=6,
        color="orange",
        line_join="bevel",
    )

# Link axes
plots[0].x_range = plots[1].x_range

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=2))

## Experimental Fano factors are greater than one

Both or theoretical analysis and the Gillespie simulation (at least of the mRNA) showed us that the Fano factor for simple gene expression is one. So, if we see Fano factors of unity in experiment, we know our model for gene transcription is at least plausible.

Gene expression in individual cells typically results in a Fano factor greater than one! There are many ways this could come about, such as explicitly considering the multiple steps involved in making a protein, or by having a switchable promoter, as you can explore in an end-of-chapter problem in forthcoming chapters. Today, we will focus on a key experimental result: **gene expression occurs in bursts.**

## Observations of bursty gene expression
In a beautiful set of experiments, [Cai, Friedman, and Xie (*Nature*, 2006)](https://doi.org/10.1038/nature04599) discovered that gene expression is bursty.  (We will come to understand what we mean by "bursty" in a moment.)

Through a clever experimental setup (we encourage you to read the paper), Cai, Friedman, and Xie were able to get accurate counts of the number of β-galactosidase molecules in individual cells over time.  They found that the number of molecules was constant over long stretches of time, and then the number suddenly increased. The figure below, taken from the paper, shows the number of β-galactosidase molecules present in a single cell over time (black) and a blank background (red). This shows that expression of the β-*gal* gene happens in bursts.  In each burst, many molecules are made, and then there is a period between bursts where no molecules are made.

<div style="margin: auto; width: 300px;">

![Cai bursts](figs/cai_bursts_1.png)

</div>

Cai and coworkers also found that the number of molecules produced per burst was geometrically distributed, as shown in the figure below.  

<div style="margin: auto; width: 300px;">

![Cai bursts 2](figs/cai_bursts_2.png)

</div>


Recall, the "story" behind the Geometric distribution.  

>We perform a series of Bernoulli trials with success probability $p_0$ until we get a success. We have $k$ failures before the success.  The probability distribution for $k$ is Geometric.

The probability mass function for the Geometric distribution is

\begin{align}
P(k;p_0) = (1-p_0)^k p.
\end{align}

Considering this story, this implies that a burst is turned on, and then it turns off by a random process after some time.

## Master equation for bursty gene expression

We also see from the experimental results tracking β-galactosidase copy number that the time scale of the bursty gene expression is much shorter than the typical decay time. So, per unit time, we can have many gene products made, not just a single gene product as we have considered thus far. So, we can re-write our transition probabilities per unit time as

\begin{align}
W(n\mid n') &= \beta' \xi_{n-n'} - \gamma (n+1) \delta_{n+1,n'},
\end{align}

where $\xi_{j}$ is the probability of making $j$ molecules in a burst and $\beta'$ is the probability per unit time of initiating production of molecules.  If $\xi_{j} = 0$ for all $j\ne 1$, then $\beta' = \beta$, and we have the same equations as before.  So, our model is only slightly changed; we just allow for more transitions in and out of state $n$.

We just have to specify $\xi_{j}$.  We know that the number of molecules produced per burst is geometrically distributed, so

\begin{align}
\xi_j = p_0(1-p_0)^j.
\end{align}

Now, our master equation is

\begin{align}
\frac{\mathrm{d}P(n,t)}{\mathrm{d}t} &= \beta'\sum_{n'=0}^{n-1}p_0(1-p_0)^{n-n'} P(n',t) + \gamma(n+1)P(n+1,t) \nonumber \\[1em]
&\;\;\;\;- \beta'\sum_{n'=n+1}^\infty p_0(1-p_0)^{n'-n}P(n,t) - \gamma n P(n,t).
\end{align}

We can simplify the second sum by noting that it can be written as a geometric series,

\begin{align}
\sum_{n'=n+1}^\infty p_0(1-p_0)^{n'-n} = p_0\sum_{j=1}^\infty (1-p_0)^j
= p_0\left(\frac{1-p_0}{1-(1-p_0)}\right) = 1-p_0.
\end{align}

Thus, we have,

\begin{align}
\frac{\mathrm{d}P(n,t)}{\mathrm{d}t} &= \beta'\sum_{n'=0}^{n-1}p_0(1-p_0)^{n-n'} P(n',t) + \gamma(n+1)P(n+1,t) \nonumber \\[1em]
&\;\;\;\;- \beta'(1-p_0)P(n,t) - \gamma n P(n,t).
\end{align}

As we might expect, an analytical solution for this master equation is difficult.  We are left to simulate it using SSA.

## The steady state distribution for bursty expression is Negative Binomial

We could derive the steady state distribution of copy numbers under the bursty master equation, but we will instead take a story matching approach to guess the solution. The story of the [Negative Binomial distribution](https://en.wikipedia.org/wiki/Negative_binomial_distribution) is as follows.

> We perform a series of Bernoulli trials with a success rate $p_0$ until we get $r$ successes. The number of failures, $n$, before we get $r$ successes is Negative Binomially distributed.

Bursty gene expression can give mRNA count distributions that are Negative Binomially distributed. Here, the Bernoulli trial is that a "failure" is production of and mRNA molecule and a "success" is that a burst in gene expression stops. So, the parameter $p_0$ is related to the length of a burst in expression (lower $p_0$ means a longer burst). The parameter $r$ is related to the frequency of the bursts. If multiple bursts are possible within the lifetime of mRNA, then $r > 1$. Then, the number of "failures" is the number of mRNA transcripts that are made in the characteristic lifetime of mRNA.

The Negative Binomial distribution is equivalent to the sum of $r$ Geometric distributions. So, the number of copies will be given by how many bursts we get before degradation. This suggests that $r = \beta'/\gamma$. We can then write the Negative Binomial probability mass function as

\begin{align}
P(n;r,p_0) = \frac{(n+r-1)!}{n!(r-1)!} p_0^r(1-p_0)^n,
\end{align}

where $r = \beta'/\gamma$.  You can plug this in to the master equation to verify that this is indeed a steady state.

We can put this in more convenient form.  Instead of using $p_0$ to parametrize the distribution, we can instead define the **burst size** $b$ as the mean of the geometric distribution describing the number transcripts in a burst,

\begin{align}
b = \frac{1-p_0}{p_0}.
\end{align}

Recall that $r$ is the typical number of bursts before degradation (or dilution), so $r$ is a **burst frequency**.  So, for convenience, we write the steady state distribution as

\begin{align}
P(n;r,b) = \frac{(n+r-1)!}{n!(r-1)!}\,\frac{b^n}{(1+b)^{n+r}}.
\end{align}

Strictly speaking, $r$ can be non-integer, so

\begin{align}
P(n;r,b) = \frac{\Gamma(n+r)}{n!\Gamma(r)!}\,\frac{b^n}{(1+b)^{n+r}},
\end{align}

where $\Gamma(x)$ is the gamma function.

The Negative Binomial is interesting because it can be peaked for $r > 1$, but has a maximum at $n=0$ for $r < 1$.  So, for a low burst frequency, we get several cells with zero copies, but we can also get cells with many. This might have interesting implications on the response of a group of cells to a rise in lactose concentration and also drop in other food sources. If $r < 1$, we then have a simple explanation for the "all-or-none" phenomenon of induction observed 60 years ago in *E. coli*  by [Novick and Weiner (*PNAS*, 1957)](https://doi.org/10.1073/pnas.43.7.553). Cells are either fully induced or not at all; with $r < 1$, many cells have no $\beta$-galactosidase at all.

The known summary statistics of the negative binomial distribution are also useful.

\begin{align}
\langle n \rangle &= rb \\[1em]
\langle n^2 \rangle &= rb(1+b+rb) \\[1em]
\sigma^2 &= rb(1+b) \\[1em]
\text{Fano factor} = F &= 1+b \\[1em]
\text{noise} = \eta &= \sqrt{\frac{1+b}{rb}}.
\end{align}

So, we can tune the burst frequency and burst size to control variability.  For example, we could keep $\langle n \rangle$ constant by increasing the burst size $b$ while decreasing the burst frequency $r$ (big, intermittent bursts), which would result in increased noise.  If, instead, we decreased the burst size while increasing the burst frequency (short, frequent bursts), we get reduced noise. This gives a design principle, **bursty gene expression enables cells to regulate the mean and cell-to-cell variability of protein levels by controlling burst frequency and burst size.**


<!-- Problem idea: Derive an ODE for the variance and noise for simple gene expression/decay. 

Another problem idea: Derive the Neg. Binomial distribution for bursty gene expression using similar arguments as used in the above for Poisson distributed.
-->

<hr>

## Problems

- [16.1: Dynamics of noise](../problems/16/problem_16.1.ipynb)
- [16.2: Noise in a switchable promoter](../problems/16/problem_16.2.ipynb)
- [16.3: A possible mechanism for bursty gene expression](../problems/16/problem_16.3.ipynb)